# Async tasks for Idea Factory

This Notebook demonstrates how to build a multi-agent system using the crewai library. The system simulates a research environment with agents acting as researchers, an integrator, and a writer, collaborating to generate and refine research and business ideas on a given topic. It leverages a large language model (LLM) for text generation and tools like web search for information gathering. Async task: the task executes asynchronously, allowing progression without waiting for completion.


🤖 **Multi agent system as a research assistant** This is a simple multi agent system with:
- Two researchers (Scientific and Business) wokring in parallel (Async) and thier proposed ideas are intergrated into a more appealing idea.  
- Memory to aid agents to remember, reason, and learn from past interactions.
- Web search to for finding a novel and impactful research contribution

By: [Ibrahim Sobh](https://www.linkedin.com/in/ibrahim-sobh-phd-8681757/)

## Install

In [1]:
%pip install -U --quiet  langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 834.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.4 MB/s eta 0:00:00


In [2]:
!pip install --quiet langchain-google-genai

In [3]:
!pip install --quiet crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## LLM

In [4]:
from IPython.display import Markdown

In [5]:
# Run this cell and paste the API key in the prompt
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5, top_p=0.85)

In [7]:
from langchain import PromptTemplate

# Chain
prompt = PromptTemplate.from_template("Tell me a very short title for a facebook post about {subject}.")
gen_chain = prompt | llm

In [28]:
# Generate
generated_txt = gen_chain.invoke({"subject": "Software testing"})
generated_txt.content

'Unveiling the Secrets of Software Testing'

## Tools

In [9]:
!pip install --upgrade --quiet  duckduckgo-search  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.2 MB/s eta 0:00:00


In [10]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

In [11]:
wrapper = DuckDuckGoSearchAPIWrapper(max_results=3)

In [12]:
from langchain_community.tools import DuckDuckGoSearchResults
web_search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)


print(web_search_tool.description)

res = web_search_tool.run("Ibrahim Sobh expert AI ")
res

A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query. Output is a JSON string array of the query results


"[snippet: Ibrahim Sobh - PhD 1y A conversation with ChatGPT about AI, study roadmap, applications, interview questions with answers, salaries, and more! Ibrahim Sobh - PhD 1y, title: Ibrahim Sobh - PhD on LinkedIn: #ai #genai #llms #codegeneration #llama, link: https://www.linkedin.com/posts/ibrahim-sobh-phd-8681757_ai-genai-llms-activity-7157819683205742594-hz9m], [snippet: Ibrahim Sobh - PhD 1y A conversation with ChatGPT about AI, study roadmap, applications, interview questions with answers, salaries, and more! Ibrahim Sobh - PhD 1y, title: Ibrahim Sobh - PhD on LinkedIn: #topvoice #machinelearning #ai # ..., link: https://www.linkedin.com/posts/ibrahim-sobh-phd-8681757_topvoice-machinelearning-deeplearning-activity-7157658639640473600-7i3M], [snippet: Before diving into AI, build a solid foundation: 1. Math and Statistics: Brush up on basic math and statistics. These skills are essential for understanding AI algorithms and models. 2 ..., title: How to Learn Artificial Intelligenc

In [13]:
from langchain.tools import Tool

tools = [
        Tool(
            name = "Search",
            func=web_search_tool.run,
            description="useful for when you need to search for research papers about AI, LLMs, and more. Input should be a search query. Output is a JSON array of the query results"
            )
        ]

## Crew


In [14]:
from crewai import Agent, Task, Process, Crew

In [29]:
Scientific_Researcher = Agent(
    role="Scientific Researcher",
    goal="Find a novel research contribution about {topic}",
    backstory="""You are a researcher in the area of {topic}. This is crucial for
		finding out novel research contributions. You are good at coming up
		with new and impactful contributions to the {topic} research.
		""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=False,  # enable collaboration between agent
    llm=llm,
    tools=tools, # web search tool
)

In [30]:
Business_Researcher = Agent(
    role="Business Researcher",
    goal="Find a novel Business idea about {topic}",
    backstory="""You are a Business researcher in the area of {topic}. This is crucial for
		finding out novel and impactful Business potential directions.
		""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=False,  # enable collaboration between agent
    llm=llm,
    tools=tools, # web search tool
)

In [31]:
Integrator = Agent(
    role="Ideas Integrator",
    goal="Integrate ideas into a more appealing one",
    backstory="""You are an expert in the {topic} covering the provides Scientific and Business aspects.
    You are able to integrate Scientific and Business ideas into a more appealing one.
		Your insights are crucial.""",
    verbose=True,
    allow_delegation=False,  # enable collaboration between agent
    llm=llm,
)

In [32]:
Writer = Agent(
    role="Technical writer",
    goal="Write a report",
    backstory="""You are a technical writer and expert in writing reports about {topic}.
		Your expertise lies in writing professional title and summary of the given topic.
		The generated text must be clear """,
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=False,  # enable collaboration between agent
    llm=llm
)

In [33]:
task1 = Task(
    description="""
    Search and propose a novel and impactful research idea in the area of {topic}.
    """,
    async_execution=True,	#If set, the task executes asynchronously, allowing progression without waiting for completion.
    agent=Scientific_Researcher,
    expected_output='A short paragraph of the new proposed research idea and its expected impact from a research point of view. If you are going to use the search tool, replace the search positional argument to be search_query'
)

task2 = Task(
    description="""
    Search and propose a novel and impactful business idea in the area of {topic}.
    """,
    async_execution=True,	#If set, the task executes asynchronously, allowing progression without waiting for completion.
    agent=Business_Researcher,
    expected_output='A short paragraph of the new proposed business idea and its expected impact from a business point of view. If you are going to use the search tool, replace the search positional argument to be search_query'
)


task3 = Task(
    description=""" Integrate proposed ideas into a more appealing one idea.
    Adressing the provided research and business point of views.
    """,
    async_execution=False,
    agent=Integrator,
    expected_output='The integrated idea as a short paragraph.'
)

task4 = Task(
    description="""Analyze and summarize the new integrated idea and write a title and abstract for it.
    The title should be professional, intersting, innovative and relevant to the area of {topic}.
    The generated abstract should adress the ideas and thier imapcts based on both research and business point of views.
		The generated text must be clear and suitable for a white papers.
    """,
    async_execution=False,
    agent=Writer,
    expected_output='Title and the abstract of the integrated idea.'
)

In [34]:
# Define the Crew
crew = Crew(
    agents=[Scientific_Researcher, Business_Researcher, Integrator, Writer],
    tasks=[task1, task2, task3, task4],
    verbose=2,
    process=Process.sequential,
    # memory=True,
    # embedder={
    #     "provider": "google",
    #     "config":{
    #         "model": 'models/embedding-001',
    #         "task_type": "retrieval_document",
    #         "title": "Embeddings for Embedchain"
    #         }
    #     }
)

# Kickoff
result = crew.kickoff(inputs={"topic": "Self-driving vehicles"})

In [35]:
Markdown(result.raw)

**Title:** The Future of Transportation: A Subscription-Based Model for Self-Driving Vehicles

**Abstract:**

The integration of self-driving vehicles and subscription-based services has the potential to revolutionize the transportation industry. This innovative business model offers several advantages for both consumers and businesses.

From a consumer perspective, subscription-based self-driving vehicles provide convenience, affordability, and accessibility. Users can summon a self-driving car on demand, eliminating the need for car ownership and maintenance. The vehicles are equipped with advanced technologies that enable them to learn and adapt to different driving environments, ensuring safety and efficiency.

For businesses, this model offers opportunities for revenue generation and customer acquisition. Subscription fees provide a recurring revenue stream, while the data collected from vehicle usage can be leveraged for product development and service enhancements. Additionally, the service can be integrated with other mobility platforms, creating a seamless and comprehensive transportation ecosystem.

The environmental impact of subscription-based self-driving vehicles is also significant. By reducing the number of vehicles on the road, this model contributes to congestion reduction, air pollution mitigation, and energy conservation. The vehicles' ability to learn and adapt to traffic patterns further optimizes their efficiency and reduces emissions.

In conclusion, the integration of self-driving vehicles and subscription-based services presents a transformative business model that addresses the evolving needs of consumers and businesses. It offers convenience, affordability, accessibility, and sustainability, while also driving innovation and economic growth in the transportation sector.